In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEVELOPMENT_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git``
    %pip install circuitsvis
    
    # PySvelte is an unmaintained visualization library, use it as a backup if circuitsvis isn't working
    # # Install another version of node that makes PySvelte work way faster
    # !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    # %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Jupyter notebook - intended for development only!


/tmp/ipykernel_250/572068249.py:21: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("load_ext autoreload")
/tmp/ipykernel_250/572068249.py:22: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic("autoreload 2")


In [2]:
# Import stuff
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.auto as tqdm
from tqdm import tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from torchtyping import TensorType as TT
from typing import List, Union, Optional
from jaxtyping import Float, Int
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML
# import circuitsvis as cv

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
from transformer_lens.utils import expand_alibi_on_query_dim

torch.set_grad_enabled(False)

def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

In [3]:
checkpoint = "smallcloudai/Refact-1_6B-fim"
kwargs = dict(
    trust_remote_code = True
)

In [4]:
# load hf model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint, **kwargs)
model

GPTRefactForCausalLM(
  (transformer): GPTRefactModel(
    (wte): Embedding(49216, 2048)
    (h): ModuleList(
      (0-31): 32 x GPTRefactBlock(
        (ln_1): LayerNormNoBias()
        (attn): Attention(
          (q): Linear(in_features=2048, out_features=2048, bias=False)
          (kv): Linear(in_features=2048, out_features=128, bias=False)
          (c_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (ln_2): LayerNormNoBias()
        (mlp): MLP(
          (gate_up_proj): Linear(in_features=2048, out_features=11264, bias=False)
          (c_proj): Linear(in_features=5632, out_features=2048, bias=False)
        )
      )
    )
  )
  (ln_f): LayerNormNoBias()
  (lm_head): Linear(in_features=2048, out_features=49216, bias=False)
)

In [5]:
# Disable folding norms and folding biases so that intermediate value
# in between transformer blocks can be compared
tl_model = HookedTransformer.from_pretrained(
    checkpoint,fold_ln=False, fold_value_biases=False, center_writing_weights=False,
    **kwargs
)
tl_model

Using pad_token, but it is not set yet.


Loaded pretrained model smallcloudai/Refact-1_6B-fim into HookedTransformer


HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_re

In [6]:
tl_model

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-31): 32 x TransformerBlock(
      (ln1): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNorm(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (hook_re

In [38]:
text = '''
TransformerLens lets you load in 50+ different open source language models,
and exposes the internal activations of the model to you. You can cache
any internal activation in the model, and add in functions to edit, remove
or replace these activations as the model runs.
'''
# text = '''
# <fim_prefix>def print_hello_world():\n    """<fim_suffix>\n    print("Hello world!")<fim_middle>
# '''
input_ids = tokenizer(text, return_tensors='pt')['input_ids']
gt_logits = model(input_ids)['logits'] # ground truth logits from hf
my_logits = tl_model(input_ids)
centered_gt_logits = gt_logits - gt_logits.mean(-1, keepdim=True)
mean_diff = (my_logits.cpu() - centered_gt_logits).mean()
print("avg logits difference:", mean_diff.item())
max_diff = (my_logits.cpu() - centered_gt_logits).abs().max()
print("max logits difference:", max_diff.item())

avg logits difference: 1.0305025170964655e-06
max logits difference: 186.43516540527344


In [39]:
gt_logits.max(dim=-1).values

tensor([[ 88.1476,  43.4963,  54.2380, 202.8922,  50.5023,  83.2911, 172.1105,
         182.6012, 101.0682, 108.3022, 192.1833, 194.4684, 125.4032, 188.9075,
         144.4984, 165.8812, 107.8797, 188.0646, 132.9209,  40.5342, 175.5600,
         174.1493, 236.4378, 176.7165, 167.8973,  54.8836, 158.3458, 153.1051,
         199.3370,  49.3762, 212.5600,   7.0188, 184.3855, 175.7485,  71.8115,
         179.1002, 183.5407,  98.9577, 110.6314, 204.5904, 140.7579,   9.6597,
          81.8662, 163.4524, 215.9029, 137.6844, 194.3499,  33.3209, 206.1470,
         153.4101, 127.5614, 190.0242, 189.7457, 241.1254, 191.4674, 208.0392,
          19.7249,  52.7732, 162.7362,  37.2505, 167.2514]])

In [37]:
gt_logits.max(dim=-1).values

tensor([[ 88.1476,  43.4963,  54.2380, 202.8922,  50.5023,  83.1182, 177.7963,
         190.5575,  99.7517, 112.9961, 195.2906, 196.8482, 152.2427, 183.8413,
         161.7404, 169.6966, 167.4622, 188.2148, 142.5233, 102.1532, 176.6607,
         175.7684, 236.2870, 163.0893, 175.3592,  73.3941, 151.9742, 156.6560,
         197.0582,  49.4439, 210.6001,   8.2312, 180.7817, 182.7687,  59.5721,
         174.9992, 180.0233,  74.3021, 137.8261, 202.9162, 106.0415,  14.3899,
          88.9807, 168.6275, 215.4047, 145.2605, 193.9221,  48.5591, 203.5147,
         138.9817, 140.6528, 187.7079, 184.0011, 241.2458, 191.1526, 207.4595,
          21.4716,  60.7495, 167.9026,  37.6301, 168.2901]])

In [8]:
gt_wte = model.transformer.wte(input_ids).cpu()
my_wte = tl_model.embed(input_ids).cpu()
print("max embed diff", (gt_wte - my_wte).abs().max().item())

max embed diff 0.0


In [9]:
gt_cache = model(input_ids, output_hidden_states=True)['hidden_states']
_, my_cache = tl_model.run_with_cache(input_ids)
atol = 1e-4
rtol = 1e-5
print("*"*5, "Matching hf and T-Lens residual stream in between transformer blocks", "*"*5)
for i in range(tl_model.cfg.n_layers):
    try:
        torch.testing.assert_close(my_cache['resid_pre',i], gt_cache[i].cuda(), atol=atol, rtol=rtol)
        print(f"layer {i} is close enough")
    except AssertionError as e:
        max_diff = (my_cache['resid_pre',i] - gt_cache[i].cuda()).abs().max()
        print(f"layer {i} \t not close, max difference: {max_diff:.2f}")
        print(e)
        break
else: 
    print(f"All layers match to atol={atol}, rtol={rtol}")

***** Matching hf and T-Lens residual stream in between transformer blocks *****
layer 0 is close enough
layer 1 	 not close, max difference: 0.48
Tensor-likes are not close!

Mismatched elements: 124459 / 124928 (99.6%)
Greatest absolute difference: 0.4802551567554474 at index (0, 12, 616) (up to 0.0001 allowed)
Greatest relative difference: 56311.15234375 at index (0, 37, 1323) (up to 1e-05 allowed)


In [10]:
my_resid_pre_0 = my_cache["resid_pre", 0].cpu()
my_embed = tl_model.embed(input_ids).cpu()
print("max embed resid_pre diff", (my_embed - my_resid_pre_0).abs().max().item())

max embed resid_pre diff 0.0


In [11]:
def _get_slopes(attn_heads: int, dev: torch.device) -> torch.Tensor:
    """
    ## Get head-specific slope $m$ for each head
    * `n_heads` is the number of heads in the attention layer $n$
    The slope for first head is
    $$\frac{1}{2^{\frac{8}{n}}} = 2^{-\frac{8}{n}}$$
    The slopes for the rest of the heads are in a geometric series with a ratio same as above.
    For instance when the number of heads is $8$ the slopes are
    $$\frac{1}{2^1}, \frac{1}{2^2}, \dots, \frac{1}{2^8}$$
    """

    # Get the closest power of 2 to `n_heads`.
    # If `n_heads` is not a power of 2, then we first calculate slopes to the closest (smaller) power of 2,
    # and then add the remaining slopes.
    n = 2 ** math.floor(math.log(attn_heads, 2))
    # $2^{-\frac{8}{n}}$
    m_0 = 2.0 ** (-8.0 / n)
    # $2^{-1\frac{8}{n}}, 2^{-2 \frac{8}{n}}, 2^{-3 \frac{8}{n}}, \dots$
    m = torch.pow(m_0, torch.arange(1, 1 + n, device=dev))

    # If `n_heads` is not a power of 2, then we add the remaining slopes.
    # We calculate the remaining slopes for $n * 2$ (avoiding slopes added previously).
    # And pick the slopes upto `n_heads`.
    if n < attn_heads:
        # $2^{-\frac{8}{2n}}$
        m_hat_0 = 2.0 ** (-4.0 / n)
        # $2^{-1\frac{8}{2n}}, 2^{-3 \frac{8}{2n}}, 2^{-5 \frac{8}{2n}}, \dots$
        # Note that we take steps by $2$ to avoid slopes added previously.
        m_hat = torch.pow(m_hat_0, torch.arange(1, 1 + 2 * (attn_heads - n), 2, device=dev))
        # Concatenate the slopes with the remaining slopes.
        m = torch.cat([m, m_hat])
    return m


def get_alibi_biases(
        B: int,
        T: int,
        attn_heads: int,
        dev: torch.device,
        dtype: torch.dtype) -> torch.Tensor:
    """
    ## Calculate the attention biases matrix
    * `n_heads` is the number of heads in the attention layer
    * `mask` is the attention mask of shape `[seq_len_q, seq_len_k]`
    This returns a matrix of shape `[seq_len_q, seq_len_k, n_heads, ]` with ALiBi attention biases.
    """

    # Get slopes $m$ for each head
    mask = torch.ones((T, T), device=dev, dtype=torch.bool)

    m = _get_slopes(attn_heads, dev).to(dtype)

    # Calculate distances $[0, 1, \dots, N]$
    # Here we calculate the distances using the mask.
    #
    # Since it's causal mask we can just use $[0, 1, \dots, N]$ too.
    # `distance = torch.arange(mask.shape[1], dtype=torch.long, device=mask.device)[None, :]`
    distance = mask.cumsum(dim=-1).to(dtype)

    # Multiply them pair-wise to get the AliBi bias matrix
    biases = distance[:, :, None] * m[None, None, :]
    biases = biases.permute(2, 0, 1)[None, :, :T, :T]
    return biases.contiguous()

In [12]:
seq_length_with_past = query_length = input_ids.shape[1]
alibi_dtype = torch.float32 if model.transformer.attention_bias_in_fp32 else model.transformer.wte.weight.dtype
gt_alibi = get_alibi_biases(
    my_resid_pre_0.shape[0], seq_length_with_past,
    model.transformer.num_heads, "cpu", alibi_dtype
)[:, :, -query_length:, :]
gt_alibi.shape

torch.Size([1, 32, 61, 61])

In [13]:
my_ln1 = tl_model.blocks[0].ln1(my_resid_pre_0.cuda()).cpu()
gt_ln1 = model.transformer.h[0].ln_1(my_resid_pre_0)
torch.testing.assert_close(my_ln1, gt_ln1, atol=1e-4, rtol=1e-5)

In [14]:
my_attn_out = my_cache["attn_out", 0].cpu()
gt_attn_out = model.transformer.h[0].attn(
    model.transformer.h[0].ln_1(my_resid_pre_0),
    alibi=gt_alibi,
)[0]
print("max attn_out diff", (my_attn_out - gt_attn_out).abs().max().item())

max attn_out diff 0.44155019521713257


In [15]:
assert tl_model.cfg.positional_embedding_type == "alibi"

In [16]:
my_q = my_cache["q", 0].cpu()
my_k = my_cache["k", 0].cpu()
my_v = my_cache["v", 0].cpu()
my_pattern = my_cache["pattern", 0].cpu()
my_z = my_cache["z", 0].cpu()

In [17]:
tl_attn = tl_model.blocks[0].attn
my_raw_scores = (
    einops.einsum(
        my_q,
        my_k,
        "batch query_pos head_index d_head, \
            batch key_pos head_index d_head \
            -> batch head_index query_pos key_pos",
    )
    / tl_attn.attn_scale
).cpu()
my_raw_scores.shape

torch.Size([1, 32, 61, 61])

In [18]:
my_alibi = tl_attn.compute_attention_linear_bias(
    batch_size=my_raw_scores.size(0),
    query_dim=my_raw_scores.size(-2),
    key_dim=my_raw_scores.size(-1),
    num_heads=tl_attn.cfg.n_heads,
    device=my_raw_scores.device,
).to(my_raw_scores.device)
my_alibi.shape

torch.Size([1, 32, 61, 61])

In [19]:
my_alibi.shape, gt_alibi.shape

(torch.Size([1, 32, 61, 61]), torch.Size([1, 32, 61, 61]))

In [20]:
my_alibi[0, 0, 0]

tensor([ 0.8409,  1.6818,  2.5227,  3.3636,  4.2045,  5.0454,  5.8863,  6.7272,
         7.5681,  8.4090,  9.2499, 10.0908, 10.9317, 11.7725, 12.6134, 13.4543,
        14.2952, 15.1361, 15.9770, 16.8179, 17.6588, 18.4997, 19.3406, 20.1815,
        21.0224, 21.8633, 22.7042, 23.5451, 24.3860, 25.2269, 26.0678, 26.9087,
        27.7496, 28.5905, 29.4314, 30.2723, 31.1132, 31.9541, 32.7950, 33.6359,
        34.4768, 35.3176, 36.1585, 36.9994, 37.8403, 38.6812, 39.5221, 40.3630,
        41.2039, 42.0448, 42.8857, 43.7266, 44.5675, 45.4084, 46.2493, 47.0902,
        47.9311, 48.7720, 49.6129, 50.4538, 51.2947])

In [21]:
my_alibi.shape

torch.Size([1, 32, 61, 61])

In [22]:
torch.testing.assert_close(my_alibi, gt_alibi, atol=1e-4, rtol=1e-5)

In [23]:
hf_attn = model.transformer.h[0].attn
hidden_states = gt_ln1
gt_query = hf_attn.q(hidden_states)
gt_kv = hf_attn.kv(hidden_states)
gt_key, gt_value = gt_kv.split(hf_attn.head_dim, dim=-1)
gt_key = gt_key.transpose(-1, -2)
gt_attn_out, gt_attn_probs = model.transformer.h[0].attn._attn(
    gt_query, gt_key, gt_value, alibi=gt_alibi
)
gt_attn_probs = einops.rearrange(
    gt_attn_probs, "batch dest head src -> batch head dest src"
)  # rearrange to match T-Lens

In [24]:
dtype = gt_query.dtype
softmax_dtype = torch.float32 if hf_attn.attention_softmax_in_fp32 else dtype
mask_value = hf_attn._get_mask_value(gt_query.device, softmax_dtype)
upcast = dtype != softmax_dtype

query_shape = gt_query.shape
batch_size = query_shape[0]
key_length = gt_key.size(-1)

# (batch_size, query_length, num_heads, head_dim) x (batch_size, head_dim, key_length)
# -> (batch_size, query_length, num_heads, key_length)
query_length = query_shape[1]
attn_shape = (batch_size, query_length, hf_attn.num_heads, key_length)
attn_view = (batch_size, query_length * hf_attn.num_heads, key_length)
# No copy needed for MQA 2, or when layer_past is provided.
query = gt_query.reshape(batch_size, query_length * hf_attn.num_heads, hf_attn.head_dim)

alibi = gt_alibi.transpose(2, 1).reshape(gt_alibi.shape[0], -1, gt_alibi.shape[-1])
initial_dtype = query.dtype
new_dtype = torch.float32 if hf_attn.attention_bias_in_fp32 else initial_dtype
gt_attn_scores = alibi.baddbmm(
    batch1=query.to(new_dtype),
    batch2=gt_key.to(new_dtype),
    beta=1,
    alpha=hf_attn.scale_factor
).view(attn_shape).to(initial_dtype)
gt_attn_scores.shape

torch.Size([1, 61, 32, 61])

In [25]:
alibi.shape, query.shape, gt_key.shape, attn_shape

(torch.Size([1, 1952, 61]),
 torch.Size([1, 1952, 64]),
 torch.Size([1, 64, 61]),
 (1, 61, 32, 61))

In [26]:
gt_attn_probs.shape, my_pattern.shape

(torch.Size([1, 32, 61, 61]), torch.Size([1, 32, 61, 61]))

In [27]:
gt_attn_probs[0, 0, 0, :].sum().item()

0.9999999403953552

In [28]:
assert np.isclose(gt_attn_probs[0, 0, 0, :].sum().item(), 1)
assert np.isclose(my_pattern[0, 1, 7, :].sum().item(), 1)

In [29]:
gt_attn_probs[0, 0, 0, :]

tensor([8.3539e-23, 7.8172e-23, 1.6393e-22, 4.8567e-22, 1.3540e-21, 1.5993e-21,
        6.2428e-21, 1.6799e-20, 2.7481e-20, 3.7070e-20, 2.2339e-19, 4.2720e-19,
        5.9115e-19, 1.7237e-18, 4.1119e-18, 7.3680e-18, 3.9440e-17, 1.3497e-16,
        9.0752e-17, 3.3818e-16, 1.0776e-15, 1.7558e-15, 3.1890e-15, 5.3143e-15,
        2.1934e-14, 7.2182e-14, 5.8982e-14, 2.2290e-13, 7.8765e-13, 2.5831e-12,
        4.8612e-12, 6.7880e-12, 1.6573e-11, 9.4055e-11, 5.1103e-11, 2.2763e-10,
        3.8827e-10, 1.3076e-09, 4.0363e-09, 3.2982e-09, 2.2943e-08, 5.5779e-08,
        4.3746e-08, 2.0308e-07, 4.3188e-07, 8.3493e-07, 1.2914e-06, 8.2610e-06,
        1.0137e-05, 6.5543e-05, 2.9327e-05, 1.1949e-04, 3.9717e-04, 6.6797e-04,
        1.1131e-03, 4.3888e-03, 1.5119e-02, 1.2354e-02, 5.0506e-02, 2.3336e-01,
        6.8185e-01])

In [30]:
my_pattern[0, 0, 0, :]

tensor([6.6476e-23, 2.0203e-22, 4.4693e-22, 1.0005e-21, 2.4869e-21, 5.5021e-21,
        1.3122e-20, 2.5021e-20, 6.5520e-20, 1.7095e-19, 3.3954e-19, 8.5492e-19,
        2.0174e-18, 4.6534e-18, 1.0808e-17, 2.4423e-17, 4.7207e-17, 1.0740e-16,
        3.1519e-16, 7.4592e-16, 1.7417e-15, 3.8459e-15, 8.8277e-15, 2.0522e-14,
        4.6332e-14, 1.1667e-13, 2.6031e-13, 6.0290e-13, 1.4467e-12, 2.5912e-12,
        7.6673e-12, 1.7205e-11, 3.8876e-11, 7.4844e-11, 2.3331e-10, 4.9860e-10,
        1.1733e-09, 2.7485e-09, 6.5240e-09, 1.4556e-08, 2.7462e-08, 7.7435e-08,
        1.8231e-07, 4.2686e-07, 9.3600e-07, 2.2583e-06, 5.1374e-06, 9.8879e-06,
        2.8244e-05, 5.2156e-05, 1.5148e-04, 3.5829e-04, 8.4296e-04, 1.8490e-03,
        4.2986e-03, 1.0294e-02, 2.4437e-02, 5.4524e-02, 1.2646e-01, 2.3410e-01,
        5.4258e-01])

In [31]:
# tl_attn.forward??

In [32]:
# hf_attn._attn??

In [ ]:
gt_attn_scores[0, :, 0, 0]

In [ ]:
torch.testing.assert_close(gt_attn_probs, my_pattern, atol=1e-4, rtol=1e-5)

In [ ]:
gt_attn_out.shape, my_z.shape

In [ ]:
# tl_model.blocks[0].attn.forward??

In [ ]:
# hf_attn??

In [ ]:
my_loss = tl_model(input_ids, return_type='loss')
print("T-Lens next token loss:", my_loss.item())
gt_outputs = model(input_ids, labels=input_ids)
gt_loss = gt_outputs.loss
print("HF next token loss:", gt_loss.item())
print("diff in loss (abs):", (gt_loss-my_loss).abs().item())